In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from collections import defaultdict

In [8]:
from models import SBN
from utils import summary, mcmc_treeprob

In [9]:
# we test the first 4 datasets here since other datasets may take large memory to compute the kl divergence.
datasets = ['DS' + str(i) for i in range(1, 5)]

In [12]:
n_rep = 10
kl_div_mat = np.empty((n_rep, 4, 5))
n_trees_mat = np.empty((n_rep, 4))
for ds_num, dataset in enumerate(datasets):
    print 'dataset {}, golden run loading ...'.format(dataset)
    tree_dict_total, tree_names_total, tree_wts_total = summary(dataset, '../data/raw_data_DS1-4/')
    taxa = tree_dict_total[tree_names_total[0]].get_leaf_names()
    emp_tree_freq = {tree_dict_total[tree_name]:tree_wts_total[i] for i, tree_name in enumerate(tree_names_total)}

    for rep in range(n_rep):
        tree_dict, tree_names, tree_wts = mcmc_treeprob('../data/short_run_data_DS1-8/' + dataset + '/rep_{}/'.format(rep+1) + dataset + '.trprobs', 'nexus')
        tree_wts = np.array(tree_wts)/sum(tree_wts)
        print dataset + ', rep {}: {} unique trees'.format(rep+1, len(tree_wts))

        model = SBN(taxa, emp_tree_freq)
        model.bn_train_prob(tree_dict, tree_names, tree_wts)

        kl_div = model.kl_div()
        kl_div_mat[rep, ds_num, 0], kl_div_mat[rep, ds_num, 1] = kl_div['ccd'], kl_div['bn']
        kl_div_mat[rep, ds_num, 4] = kl_div['freq']

        print 'running EM >>>>>>'
        logp = model.bn_em_prob(tree_dict, tree_names, tree_wts, maxiter=1000, abstol=1e-05, monitor=True, MAP=False)
        kl_div_mat[rep, ds_num, 2] = model.kl_div(method='bn')['bn']

        print 'running EM-alpha >>>>>>'
        model = SBN(taxa, emp_tree_freq, alpha=0.0001)
        logp = model.bn_em_prob(tree_dict, tree_names, tree_wts, maxiter=1000, abstol=1e-05, monitor=True, MAP=True)
        kl_div_mat[rep, ds_num, 3] = model.kl_div(method='bn', MAP=True)['bn']

        n_trees_mat[rep, ds_num] = len(tree_wts)

        print dataset + ', rep {}'.format(rep+1)
        print 'kl_div: {}'.format(kl_div_mat[rep, ds_num, :]) 

dataset DS1, golden run loading ...
DS1, rep 1: 1278 unique trees
running EM >>>>>>
Iter 1: current per tree log-likelihood -2.960113
Iter 2: current per tree log-likelihood -2.955872
Iter 3: current per tree log-likelihood -2.954958
Iter 4: current per tree log-likelihood -2.954789
Iter 5: current per tree log-likelihood -2.954687
Iter 6: current per tree log-likelihood -2.954600
Iter 7: current per tree log-likelihood -2.954514
Iter 8: current per tree log-likelihood -2.954425
Iter 9: current per tree log-likelihood -2.954337
Iter 10: current per tree log-likelihood -2.954265
Iter 11: current per tree log-likelihood -2.954215
Iter 12: current per tree log-likelihood -2.954179
Iter 13: current per tree log-likelihood -2.954148
Iter 14: current per tree log-likelihood -2.954115
Iter 15: current per tree log-likelihood -2.954074
Iter 16: current per tree log-likelihood -2.954017
Iter 17: current per tree log-likelihood -2.953934
Iter 18: current per tree log-likelihood -2.953806
Iter 19

Iter 26: current per tree log-likelihood -2.937244
Iter 27: current per tree log-likelihood -2.932259
Iter 28: current per tree log-likelihood -2.927065
Iter 29: current per tree log-likelihood -2.922438
Iter 30: current per tree log-likelihood -2.918847
Iter 31: current per tree log-likelihood -2.916253
Iter 32: current per tree log-likelihood -2.914314
Iter 33: current per tree log-likelihood -2.912691
Iter 34: current per tree log-likelihood -2.911178
Iter 35: current per tree log-likelihood -2.909700
Iter 36: current per tree log-likelihood -2.908254
Iter 37: current per tree log-likelihood -2.906876
Iter 38: current per tree log-likelihood -2.905609
Iter 39: current per tree log-likelihood -2.904487
Iter 40: current per tree log-likelihood -2.903532
Iter 41: current per tree log-likelihood -2.902747
Iter 42: current per tree log-likelihood -2.902122
Iter 43: current per tree log-likelihood -2.901637
Iter 44: current per tree log-likelihood -2.901270
Iter 45: current per tree log-l

Iter 53: current per tree log-likelihood -2.858090
Iter 54: current per tree log-likelihood -2.858053
Iter 55: current per tree log-likelihood -2.858021
Iter 56: current per tree log-likelihood -2.857992
Iter 57: current per tree log-likelihood -2.857967
Iter 58: current per tree log-likelihood -2.857945
Iter 59: current per tree log-likelihood -2.857924
Iter 60: current per tree log-likelihood -2.857906
Iter 61: current per tree log-likelihood -2.857889
Iter 62: current per tree log-likelihood -2.857874
Iter 63: current per tree log-likelihood -2.857860
Iter 64: current per tree log-likelihood -2.857848
Iter 65: current per tree log-likelihood -2.857836
Iter 66: current per tree log-likelihood -2.857826
Iter 67: current per tree log-likelihood -2.857816
running EM-alpha >>>>>>
Iter 1: current per tree log-likelihood -2.912965
Iter 2: current per tree log-likelihood -2.908754
Iter 3: current per tree log-likelihood -2.907982
Iter 4: current per tree log-likelihood -2.907841
Iter 5: cur

Iter 12: current per tree log-likelihood -2.984157
Iter 13: current per tree log-likelihood -2.984101
Iter 14: current per tree log-likelihood -2.984046
Iter 15: current per tree log-likelihood -2.983984
Iter 16: current per tree log-likelihood -2.983899
Iter 17: current per tree log-likelihood -2.983775
Iter 18: current per tree log-likelihood -2.983584
Iter 19: current per tree log-likelihood -2.983285
Iter 20: current per tree log-likelihood -2.982817
Iter 21: current per tree log-likelihood -2.982081
Iter 22: current per tree log-likelihood -2.980931
Iter 23: current per tree log-likelihood -2.979164
Iter 24: current per tree log-likelihood -2.976537
Iter 25: current per tree log-likelihood -2.972847
Iter 26: current per tree log-likelihood -2.968105
Iter 27: current per tree log-likelihood -2.962727
Iter 28: current per tree log-likelihood -2.957489
Iter 29: current per tree log-likelihood -2.953121
Iter 30: current per tree log-likelihood -2.949889
Iter 31: current per tree log-l

Iter 40: current per tree log-likelihood -2.920533
Iter 41: current per tree log-likelihood -2.919705
Iter 42: current per tree log-likelihood -2.919036
Iter 43: current per tree log-likelihood -2.918510
Iter 44: current per tree log-likelihood -2.918106
Iter 45: current per tree log-likelihood -2.917800
Iter 46: current per tree log-likelihood -2.917571
Iter 47: current per tree log-likelihood -2.917399
Iter 48: current per tree log-likelihood -2.917269
Iter 49: current per tree log-likelihood -2.917169
Iter 50: current per tree log-likelihood -2.917090
Iter 51: current per tree log-likelihood -2.917027
Iter 52: current per tree log-likelihood -2.916974
Iter 53: current per tree log-likelihood -2.916930
Iter 54: current per tree log-likelihood -2.916892
Iter 55: current per tree log-likelihood -2.916859
Iter 56: current per tree log-likelihood -2.916830
Iter 57: current per tree log-likelihood -2.916805
Iter 58: current per tree log-likelihood -2.916782
Iter 59: current per tree log-l

...

Iter 93: current per tree log-likelihood -3.515846
Iter 94: current per tree log-likelihood -3.515821
Iter 95: current per tree log-likelihood -3.515798
Iter 96: current per tree log-likelihood -3.515776
Iter 97: current per tree log-likelihood -3.515755
Iter 98: current per tree log-likelihood -3.515735
Iter 99: current per tree log-likelihood -3.515716
Iter 100: current per tree log-likelihood -3.515698
Iter 101: current per tree log-likelihood -3.515681
Iter 102: current per tree log-likelihood -3.515665
Iter 103: current per tree log-likelihood -3.515650
Iter 104: current per tree log-likelihood -3.515635
Iter 105: current per tree log-likelihood -3.515621
Iter 106: current per tree log-likelihood -3.515608
Iter 107: current per tree log-likelihood -3.515595
Iter 108: current per tree log-likelihood -3.515582
Iter 109: current per tree log-likelihood -3.515571
Iter 110: current per tree log-likelihood -3.515559
Iter 111: current per tree log-likelihood -3.515548
Iter 112: current p

Iter 19: current per tree log-likelihood -3.906176
Iter 20: current per tree log-likelihood -3.905948
Iter 21: current per tree log-likelihood -3.905638
Iter 22: current per tree log-likelihood -3.905179
Iter 23: current per tree log-likelihood -3.904459
Iter 24: current per tree log-likelihood -3.903310
Iter 25: current per tree log-likelihood -3.901507
Iter 26: current per tree log-likelihood -3.898840
Iter 27: current per tree log-likelihood -3.895248
Iter 28: current per tree log-likelihood -3.890944
Iter 29: current per tree log-likelihood -3.886346
Iter 30: current per tree log-likelihood -3.881861
Iter 31: current per tree log-likelihood -3.877735
Iter 32: current per tree log-likelihood -3.874059
Iter 33: current per tree log-likelihood -3.870844
Iter 34: current per tree log-likelihood -3.868068
Iter 35: current per tree log-likelihood -3.865693
Iter 36: current per tree log-likelihood -3.863657
Iter 37: current per tree log-likelihood -3.861919
Iter 38: current per tree log-l

Iter 46: current per tree log-likelihood -3.855810
Iter 47: current per tree log-likelihood -3.855506
Iter 48: current per tree log-likelihood -3.855221
Iter 49: current per tree log-likelihood -3.854950
Iter 50: current per tree log-likelihood -3.854691
Iter 51: current per tree log-likelihood -3.854445
Iter 52: current per tree log-likelihood -3.854212
Iter 53: current per tree log-likelihood -3.853993
Iter 54: current per tree log-likelihood -3.853788
Iter 55: current per tree log-likelihood -3.853598
Iter 56: current per tree log-likelihood -3.853420
Iter 57: current per tree log-likelihood -3.853255
Iter 58: current per tree log-likelihood -3.853100
Iter 59: current per tree log-likelihood -3.852955
Iter 60: current per tree log-likelihood -3.852819
Iter 61: current per tree log-likelihood -3.852690
Iter 62: current per tree log-likelihood -3.852569
Iter 63: current per tree log-likelihood -3.852455
Iter 64: current per tree log-likelihood -3.852346
Iter 65: current per tree log-l

Iter 68: current per tree log-likelihood -3.513642
Iter 69: current per tree log-likelihood -3.513565
Iter 70: current per tree log-likelihood -3.513494
Iter 71: current per tree log-likelihood -3.513430
Iter 72: current per tree log-likelihood -3.513370
Iter 73: current per tree log-likelihood -3.513315
Iter 74: current per tree log-likelihood -3.513263
Iter 75: current per tree log-likelihood -3.513215
Iter 76: current per tree log-likelihood -3.513171
Iter 77: current per tree log-likelihood -3.513129
Iter 78: current per tree log-likelihood -3.513089
Iter 79: current per tree log-likelihood -3.513052
Iter 80: current per tree log-likelihood -3.513017
Iter 81: current per tree log-likelihood -3.512983
Iter 82: current per tree log-likelihood -3.512951
Iter 83: current per tree log-likelihood -3.512920
Iter 84: current per tree log-likelihood -3.512891
Iter 85: current per tree log-likelihood -3.512862
Iter 86: current per tree log-likelihood -3.512835
Iter 87: current per tree log-l

Iter 52: current per tree log-likelihood -3.517119
Iter 53: current per tree log-likelihood -3.516829
Iter 54: current per tree log-likelihood -3.516564
Iter 55: current per tree log-likelihood -3.516324
Iter 56: current per tree log-likelihood -3.516105
Iter 57: current per tree log-likelihood -3.515907
Iter 58: current per tree log-likelihood -3.515728
Iter 59: current per tree log-likelihood -3.515567
Iter 60: current per tree log-likelihood -3.515421
Iter 61: current per tree log-likelihood -3.515289
Iter 62: current per tree log-likelihood -3.515169
Iter 63: current per tree log-likelihood -3.515061
Iter 64: current per tree log-likelihood -3.514963
Iter 65: current per tree log-likelihood -3.514873
Iter 66: current per tree log-likelihood -3.514791
Iter 67: current per tree log-likelihood -3.514716
Iter 68: current per tree log-likelihood -3.514647
Iter 69: current per tree log-likelihood -3.514583
Iter 70: current per tree log-likelihood -3.514523
Iter 71: current per tree log-l

In [13]:
kl_div_mean = np.mean(kl_div_mat, axis=0)

In [14]:
kl_div_mean

array([[0.60270922, 0.06867027, 0.01360177, 0.01301135, 0.0154867 ],
       [0.0218376 , 0.02184212, 0.01986416, 0.01279808, 0.01222526],
       [0.20735418, 0.11515626, 0.12434555, 0.0881886 , 0.35389013],
       [0.19517261, 0.10210946, 0.07632767, 0.06370093, 0.53218207]])